In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import collections, math

import matplotlib.pyplot as plt
%matplotlib inline 

In [ ]:
#crop test images by detections_full_AGNOSTICnms.pkl 取同类中score最高的，按类的数量加权平均

RFCN_MODEL = 'resnet101_rfcn_ohem_iter_30000'
FISH_CLASSES = ['NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']

import pickle 
with open('../data/RFCN_detections/detections_full_AGNOSTICnms_'+RFCN_MODEL+'.pkl','rb') as f:
    detections_full_AGNOSTICnms = pickle.load(f, encoding='latin1') 
    
CONF_THRESH = 0.8
outputs = []
count = np.zeros(len(detections_full_AGNOSTICnms))

for im in range(len(detections_full_AGNOSTICnms)):
    if im%1000 == 0:
        print(im)
    detects_im = detections_full_AGNOSTICnms[im]
    score_max = np.max(detects_im[:,5:], axis=1)
    inds = np.argmax(detects_im[:,5:], axis=1)
    labels = [FISH_CLASSES[ind+1] for ind in inds]
    columns = ['box1', 'box2', 'box3', 'box4']
    columns.extend(FISH_CLASSES)
    detects_im_df = pd.DataFrame(detects_im, columns=columns)
    detects_im_df['max_cls'] = labels
    detects_im_df['max_score'] = score_max
    detects_im_df = detects_im_df[detects_im_df["max_score"]>=CONF_THRESH]  
    detects_im_df['Counts'] = detects_im_df.groupby(['max_cls'])['max_cls'].transform('count')
    idx = detects_im_df.groupby(['max_cls'])['max_score'].transform(max) == detects_im_df['max_score']
    detects_im_df = detects_im_df[idx]
    detects_im_array = detects_im_df[columns].as_matrix()
    count[im] = detects_im_array.shape[0]
    if detects_im_array.shape[0] == 0:
        ind = np.argmax(score_max)
        outputs_im = detects_im[[ind], :]
    else:
        outputs_im = detects_im_array
    outputs.append(outputs_im)

print(sum([outputs[i].shape[0] for i in range(len(outputs))]))

In [ ]:
count8 = count

In [ ]:
collections.Counter(count)

In [ ]:
np.where(count==2)[0]
#np.ndarray.tolist(np.where(count==2)[0][:10])

In [ ]:
diff = [(count5[i]-count9[i])>0 and count9[i]!=0 for i in range(4777)]
ims_diff = []
for i in range(1,2):
    if diff[i] == True:
        ims_diff.append(i)

ims_diff

In [ ]:
#visualize test image crop
FISH_CLASSES = ['NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']

with open("../RFCN/ImageSets/Main/test.txt","r") as f:
    ims = f.readlines()
test_files = [im[:-1]+'.jpg' for im in ims]

for j in np.ndarray.tolist(np.where(count==0)[0][-10:]):
    dets = outputs[j]
    print(dets)
    im = Image.open("../RFCN/JPEGImages/"+test_files[j])
    im = np.asarray(im)
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.imshow(im, aspect='equal')
    for i in range(dets.shape[0]):
        bbox = dets[i, :4]
        score = np.amax(dets[i,4:])
        index = np.argmax(dets[i,4:])
        class_name = FISH_CLASSES[index]
        #if not (bbox[0] == 0 and bbox[1] == 0 and bbox[2] == 0 and bbox[3] == 0):
        ax.add_patch(plt.Rectangle((bbox[0], bbox[1]),
                          bbox[2] - bbox[0],
                          bbox[3] - bbox[1], fill=False,
                          edgecolor='red', linewidth=3.5))
        ax.text(bbox[0], bbox[1] - 2,
                '{:s} {:.3f}'.format(class_name, score),
                bbox=dict(facecolor='blue', alpha=0.5),
                fontsize=14, color='white')

    ax.set_title(('Image {} detections with '
                  'p({} | box) >= {:.1f}').format(j, class_name, CONF_THRESH),fontsize=14)
    plt.axis('off')
    plt.tight_layout()
    plt.draw()

In [4]:
#submission from detections_full_AGNOSTICnms.pkl 取同类中score最高的，按类的数量加权平均

RFCN_MODEL = 'resnet101_rfcn_ohem_iter_30000'
FISH_CLASSES = ['NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']

import pickle 
with open('../data/RFCN_detections/detections_full_AGNOSTICnms_'+RFCN_MODEL+'.pkl','rb') as f:
    detections_full_AGNOSTICnms = pickle.load(f, encoding='latin1') 
    
CONF_THRESH = 0.9
outputs = np.ndarray((len(detections_full_AGNOSTICnms), len(FISH_CLASSES)), dtype=np.float64)

for im in range(len(detections_full_AGNOSTICnms)):
    if im%1000 == 0:
        print(im)
    detects_im = detections_full_AGNOSTICnms[im]
    score_max = np.max(detects_im[:,5:], axis=1)
    inds = np.argmax(detects_im[:,5:], axis=1)
    labels = [FISH_CLASSES[ind+1] for ind in inds]
    columns = ['box1', 'box2', 'box3', 'box4']
    columns.extend(FISH_CLASSES)
    detects_im_df = pd.DataFrame(detects_im, columns=columns)
    detects_im_df['max_cls'] = labels
    detects_im_df['max_score'] = score_max
    detects_im_df = detects_im_df[detects_im_df["max_score"]>=CONF_THRESH]  
    detects_im_df['Counts'] = detects_im_df.groupby(['max_cls'])['max_cls'].transform('count')
    idx = detects_im_df.groupby(['max_cls'])['max_score'].transform(max) == detects_im_df['max_score']
    detects_im_df = detects_im_df[idx]
    l = FISH_CLASSES.copy()
    l.append('Counts')
    detects_im_array = detects_im_df[l].as_matrix()
    if detects_im_array.shape[0] == 0:
        ind = np.argmax(score_max)
        outputs_im = detects_im[ind,4:12]
    else:
        outputs_im = np.average(detects_im_array[:,:-1], axis=0, weights=detects_im_array[:,-1], returned=False)
    outputs[im] = outputs_im
    


0
1000
2000
3000
4000


In [5]:
# CLIP_THRESH = 0.02
# outputs_cliped = np.clip(outputs, CLIP_THRESH, 1, out=None)
# outputs_cliped = outputs_cliped/np.sum(outputs_cliped, axis=1, keepdims=True)
# test_preds = outputs_cliped[:1000]
# train_preds = outputs_cliped[1000:]

#temperature
T = 2.5
outputs_T = np.exp(np.log(outputs)/T)
outputs_T = outputs_T/np.sum(outputs_T, axis=1, keepdims=True)
test_preds = outputs_T[:1000]
train_preds = outputs_T[1000:]


with open("../RFCN/ImageSets/Main/test.txt","r") as f:
    ims = f.readlines()
test_files = [im[:-1]+'.jpg' for im in ims[:1000]]
train_files = [im[:-1] for im in ims[1000:]]
with open("../RFCN/ImageSets/Main/train_test.txt","r") as f:
    train_file_labels = f.readlines()

log_losses = []
for i in range(len(train_preds)):
    im = train_files[i]
    for im_label in train_file_labels:
        if im == im_label[:9]:
            label = im_label[10:-1]
            index = FISH_CLASSES.index(label)
            log_losses.append(-math.log(train_preds[i][index]))
log_loss = sum(log_losses) / float(len(log_losses))
print('logloss of train is', log_loss )

import datetime

submission = pd.DataFrame(test_preds, columns=FISH_CLASSES)
#submission.loc[:, 'image'] = pd.Series(test_files, index=submission.index)
submission.insert(0, 'image', test_files)

now = datetime.datetime.now()
# info = 'RFCN_AGONOSTICnms_clsMaxAve_resnet101_rfcn_ohem_iter_30000_conf{:.1f}_'.format(CONF_THRESH) + 'clip{:.2f}_'.format(CLIP_THRESH) + '{:.4f}'.format(log_loss)
info = 'RFCN_AGONOSTICnms_clsMaxAve_'+RFCN_MODEL+'_conf{:.2f}_T{}_'.format(CONF_THRESH, T) + '{:.4f}'.format(log_loss)
sub_file = 'submission_' + info + '.csv'
submission.to_csv(sub_file, index=False)

logloss of train is 0.1125375372472951
